In [20]:
import pandas as pd

df = pd.read_csv("../../Data/m4_imputed.csv")


In [21]:
median_wl_time = df['wl_time'].median()
print(f"Median waiting list time: {median_wl_time}")

df['wl_time'] = df['wl_time'].apply(lambda x: 0 if x < median_wl_time else 1)
print(df['wl_time'].value_counts(normalize=True))

Median waiting list time: 78.0
1    0.501886
0    0.498114
Name: wl_time, dtype: float64


In [22]:
import keras.backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [23]:
from sklearn.model_selection import train_test_split
from autokeras import StructuredDataClassifier
from sklearn.metrics import f1_score

X = df.drop(columns=['wl_time'])
y = df['wl_time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = StructuredDataClassifier(max_trials=25,
                               project_name="m4_pre_feat",
                               metrics=["accuracy"],
                               overwrite=False)

clf.fit(X_train, y_train, verbose=1)
print(clf.evaluate(X_test, y_test))
f1_s = f1_score(y_test, clf.predict(X_test))
print(f"F1 score: {f1_s}")
print("F1 Macro:", f1_score(y_test, clf.predict(X_test), average='macro'))


INFO:tensorflow:Reloading Tuner from ./m4_pre_feat/tuner0.json


INFO:tensorflow:Reloading Tuner from ./m4_pre_feat/tuner0.json



Search: Running Trial #15

Value             |Best Value So Far |Hyperparameter
False             |False             |structured_data_block_1/normalize
True              |True              |structured_data_block_1/dense_block_1/use_batchnorm
3                 |2                 |structured_data_block_1/dense_block_1/num_layers
32                |512               |structured_data_block_1/dense_block_1/units_0
0.25              |0                 |structured_data_block_1/dense_block_1/dropout
128               |32                |structured_data_block_1/dense_block_1/units_1
0                 |0                 |classification_head_1/dropout
adam_weight_decay |adam              |optimizer
2e-05             |0.001             |learning_rate



KeyboardInterrupt: 